In [1]:
# In file dependencies 
import json
import requests
import time
import pandas as pd
import numpy as np
import datetime
from config import api_key
from config import password

# database dependencies
import psycopg2
import sqlite3

In [2]:
# connection to database
conn = psycopg2.connect(database="world_happiness_index", user = "postgres", password = password, host = "127.0.0.1", port = "5432")

In [3]:

# empty list to hold list of countries
country_list = list()

# accessing database to pull all countries from all tables
cur = conn.cursor()
cur.execute(f'''SELECT DISTINCT year_2015.country AS YEAR2015, year_2016.country AS YEAR2016, year_2017.country AS YEAR2017, year_2018.country AS YEAR2018, year_2019.country AS YEAR2019
FROM year_2015
JOIN year_2016
ON	year_2015.country = year_2016.country
JOIN year_2017
ON	year_2015.country = year_2017.country
JOIN year_2018
ON	year_2015.country = year_2018.country
JOIN year_2019
ON	year_2015.country = year_2019.country; ''')

rows = cur.fetchall()

for row in rows:
    for item in row:
        country_list.append(item)
    
cur.close()

# removing duplicates from list
country_list = list(dict.fromkeys(country_list))

country_list

['Afghanistan',
 'Albania',
 'Algeria',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Belgium',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 'Croatia',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Estonia',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Guatemala',
 'Guinea',
 'Haiti',
 'Honduras',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Ivory Coast',
 'Jamaica',
 'Japan',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lebanon',
 'Liberia',
 'Libya',
 'Lithuania',
 'Luxembourg',
 'Madagascar'

In [4]:
# testing api pull from google geo locations


#creating api/config information
url = 'https://maps.googleapis.com/maps/api/geocode/json?address='

#build query URL
query_url = url+"peru"+"&key="+api_key

#checking format
# print(query_url)

#testing pull from api
response = requests.get(query_url).json()
# testing index
response["results"][0]["geometry"]["location"]["lng"]

# d = response["results"][0]["geometry"]["location"]

# d

-75.015152

In [5]:

#Pulling data via API

#creating empty lists for variables returned from API
lat = []
lng = []
# country = []
country_list_2 =[]

# counter + row tracker 
count = 0
set_num = 1


#looping through data to get lat and lng data for cities
print('Beginning Data Retrieval')
print('-----------------------------')

for country in country_list:
    #adding try logic to code to keep it moving if no result
    try:
        response = requests.get(url+country+"&key="+api_key).json()
        lat.append(response["results"][0]["geometry"]["location"]["lat"])
        lng.append(response["results"][0]["geometry"]["location"]["lng"])
#         country.append(response['sys']['country'])
        
        #if logic to track where we are in data set and eliminate duplicates
        if count >= 49:
            count = 1
            set_num = set_num + 1
            country_list_2.append(country)
            
        else:
            count = count + 1
            country_list_2.append(country)
            
        #displaying where we are in data set and what city was processed
        print(f'Processing record {count} of set {set_num} | {country}')
        #exception value for row if not found
    except Exception:
        print('Country not found. skipping ...')

#confirming looped through all data in set
print('-----------------------------')
print('data retrieval complete')

Beginning Data Retrieval
-----------------------------
Processing record 1 of set 1 | Afghanistan
Processing record 2 of set 1 | Albania
Processing record 3 of set 1 | Algeria
Processing record 4 of set 1 | Argentina
Processing record 5 of set 1 | Armenia
Processing record 6 of set 1 | Australia
Processing record 7 of set 1 | Austria
Processing record 8 of set 1 | Azerbaijan
Processing record 9 of set 1 | Bahrain
Processing record 10 of set 1 | Bangladesh
Processing record 11 of set 1 | Belarus
Processing record 12 of set 1 | Belgium
Processing record 13 of set 1 | Benin
Processing record 14 of set 1 | Bhutan
Processing record 15 of set 1 | Bolivia
Processing record 16 of set 1 | Bosnia and Herzegovina
Processing record 17 of set 1 | Botswana
Processing record 18 of set 1 | Brazil
Processing record 19 of set 1 | Bulgaria
Processing record 20 of set 1 | Burkina Faso
Processing record 21 of set 1 | Burundi
Processing record 22 of set 1 | Cambodia
Processing record 23 of set 1 | Cameroon


In [6]:

#creating dictionary with data pulled from api
country_dict={"country": country_list_2,
              "lat": lat,
              "lng": lng}

country_df= pd.DataFrame(country_dict)

 
print(country_df.shape)
country_df.head()

(146, 3)


,country,lat,lng
0,Afghanistan,33.939110,67.709953
1,Albania,41.153332,20.168331
2,Algeria,28.033886,1.659626
3,Argentina,-38.416097,-63.616672
4,Armenia,40.069099,45.038189


In [15]:
# pulling from postgre database for 2015

hr_2015 = []
ct_2015 = []

cur = conn.cursor()

cur.execute('''SELECT country, happiness_rank FROM year_2015;''')

rows = cur.fetchall()

for row in rows:
    
    ct_2015.append(row[0])
    print(row[0])
    
    hr_2015.append(row[1])
    print(row[1])
    
cur.close()

Afghanistan
153
Albania
95
Algeria
68
Argentina
30
Armenia
127
Australia
10
Austria
13
Azerbaijan
80
Bahrain
49
Bangladesh
109
Belarus
59
Belgium
19
Benin
155
Bhutan
79
Bolivia
51
Bosnia and Herzegovina
96
Botswana
128
Brazil
16
Bulgaria
134
Burkina Faso
152
Burundi
157
Cambodia
145
Cameroon
133
Canada
5
Chad
149
Chile
27
China
84
Colombia
33
Congo (Brazzaville)
139
Congo (Kinshasa)
120
Costa Rica
12
Croatia
62
Cyprus
67
Czech Republic
31
Denmark
3
Dominican Republic
98
Ecuador
48
Egypt
135
El Salvador
42
Estonia
73
Ethiopia
122
Finland
6
France
29
Gabon
143
Georgia
130
Germany
26
Ghana
114
Greece
102
Guatemala
43
Guinea
150
Haiti
119
Honduras
105
Hong Kong
72
Hungary
104
Iceland
2
India
117
Indonesia
74
Iran
110
Iraq
112
Ireland
18
Israel
11
Italy
50
Ivory Coast
151
Jamaica
65
Japan
46
Jordan
82
Kazakhstan
54
Kenya
125
Kosovo
69
Kuwait
39
Kyrgyzstan
77
Latvia
89
Lebanon
103
Liberia
116
Libya
63
Lithuania
56
Luxembourg
17
Madagascar
147
Malawi
131
Malaysia
61
Mali
138
Malta
37
Mauritan

In [16]:
# pulling from postgre database for 2016

hr_2016 = []
ct_2016 = []

cur = conn.cursor()

cur.execute('''SELECT country, happiness_rank FROM year_2016;''')

rows = cur.fetchall()

for row in rows:
    
    ct_2016.append(row[0])
    print(row[0])
    
    hr_2016.append(row[1])
    print(row[1])
    
cur.close()

Afghanistan
154
Albania
109
Algeria
38
Argentina
26
Armenia
121
Australia
9
Austria
12
Azerbaijan
81
Bahrain
42
Bangladesh
110
Belarus
61
Belgium
18
Benin
153
Bhutan
84
Bolivia
59
Bosnia and Herzegovina
87
Botswana
137
Brazil
17
Bulgaria
129
Burkina Faso
145
Burundi
157
Cambodia
140
Cameroon
114
Canada
6
Chad
144
Chile
24
China
83
Colombia
31
Congo (Brazzaville)
127
Congo (Kinshasa)
125
Costa Rica
14
Croatia
74
Cyprus
69
Czech Republic
27
Denmark
1
Dominican Republic
89
Ecuador
51
Egypt
120
El Salvador
46
Estonia
72
Ethiopia
115
Finland
5
France
32
Gabon
134
Georgia
126
Germany
16
Ghana
124
Greece
99
Guatemala
39
Guinea
151
Haiti
136
Honduras
104
Hong Kong
75
Hungary
91
Iceland
3
India
118
Indonesia
79
Iran
105
Iraq
112
Ireland
19
Israel
11
Italy
50
Ivory Coast
139
Jamaica
73
Japan
53
Jordan
80
Kazakhstan
54
Kenya
122
Kosovo
77
Kuwait
41
Kyrgyzstan
85
Latvia
68
Lebanon
93
Liberia
150
Libya
67
Lithuania
60
Luxembourg
20
Madagascar
148
Malawi
132
Malaysia
47
Mali
135
Malta
30
Mauritania


In [17]:
# pulling from postgre database for 2017

hr_2017 = []
ct_2017 = []

cur = conn.cursor()

cur.execute('''SELECT country, happiness_rank FROM year_2017;''')

rows = cur.fetchall()

for row in rows:
    
    ct_2017.append(row[0])
    print(row[0])
    
    hr_2017.append(row[1])
    print(row[1])
    
cur.close()

Afghanistan
141
Albania
109
Algeria
53
Argentina
24
Armenia
121
Australia
10
Austria
13
Azerbaijan
85
Bahrain
41
Bangladesh
110
Belarus
67
Belgium
17
Benin
143
Bhutan
97
Bolivia
58
Bosnia and Herzegovina
90
Botswana
142
Brazil
22
Bulgaria
105
Burkina Faso
134
Burundi
154
Cambodia
129
Cameroon
107
Canada
7
Chad
137
Chile
20
China
79
Colombia
36
Congo (Brazzaville)
124
Congo (Kinshasa)
126
Costa Rica
12
Croatia
77
Cyprus
65
Czech Republic
23
Denmark
2
Dominican Republic
86
Ecuador
44
Egypt
104
El Salvador
45
Estonia
66
Ethiopia
119
Finland
5
France
31
Gabon
118
Georgia
125
Germany
16
Ghana
131
Greece
87
Guatemala
29
Guinea
149
Haiti
145
Honduras
91
Hong Kong
71
Hungary
75
Iceland
3
India
122
Indonesia
81
Iran
108
Iraq
117
Ireland
15
Israel
11
Italy
48
Ivory Coast
128
Jamaica
76
Japan
51
Jordan
74
Kazakhstan
60
Kenya
112
Kosovo
78
Kuwait
39
Kyrgyzstan
98
Latvia
54
Lebanon
88
Liberia
148
Libya
68
Lithuania
52
Luxembourg
18
Madagascar
144
Malawi
136
Malaysia
42
Mali
127
Malta
27
Mauritania


In [18]:
# pulling from postgre database for 2018

hr_2018 = []
ct_2018 = []

cur = conn.cursor()

cur.execute('''SELECT country, happiness_rank FROM year_2018;''')

rows = cur.fetchall()

for row in rows:
    
    ct_2018.append(row[0])
    print(row[0])
    
    hr_2018.append(row[1])
    print(row[1])
    
cur.close()

Afghanistan
145
Albania
112
Algeria
84
Argentina
29
Armenia
129
Australia
10
Austria
12
Azerbaijan
87
Bahrain
43
Bangladesh
115
Belarus
73
Belgium
16
Benin
136
Bhutan
97
Bolivia
62
Bosnia and Herzegovina
93
Botswana
146
Brazil
28
Bulgaria
100
Burkina Faso
121
Burundi
156
Cambodia
120
Cameroon
99
Canada
7
Chad
131
Chile
25
China
86
Colombia
37
Congo (Brazzaville)
114
Congo (Kinshasa)
132
Costa Rica
13
Croatia
82
Cyprus
61
Czech Republic
21
Denmark
3
Dominican Republic
83
Ecuador
48
Egypt
122
El Salvador
40
Estonia
63
Ethiopia
127
Finland
1
France
23
Gabon
103
Georgia
128
Germany
15
Ghana
108
Greece
79
Guatemala
30
Guinea
140
Haiti
148
Honduras
72
Hong Kong
76
Hungary
69
Iceland
4
India
133
Indonesia
96
Iran
106
Iraq
117
Ireland
14
Israel
19
Italy
47
Ivory Coast
107
Jamaica
56
Japan
54
Jordan
90
Kazakhstan
60
Kenya
124
Kosovo
66
Kuwait
45
Kyrgyzstan
92
Latvia
53
Lebanon
80
Liberia
149
Libya
70
Lithuania
50
Luxembourg
17
Madagascar
143
Malawi
147
Malaysia
35
Mali
118
Malta
22
Mauritania
1

In [19]:
# pulling from postgre database for 2019

hr_2019 = []
ct_2019 = []

cur = conn.cursor()

cur.execute('''SELECT country, happiness_rank FROM year_2019;''')

rows = cur.fetchall()

for row in rows:
    
    ct_2019.append(row[0])
    print(row[0])
    
    hr_2019.append(row[1])
    print(row[1])
    
cur.close()

Afghanistan
154
Albania
107
Algeria
88
Argentina
47
Armenia
116
Australia
11
Austria
10
Azerbaijan
90
Bahrain
37
Bangladesh
125
Belarus
81
Belgium
18
Benin
102
Bhutan
95
Bolivia
61
Bosnia and Herzegovina
78
Botswana
148
Brazil
32
Bulgaria
97
Burkina Faso
115
Burundi
145
Cambodia
109
Cameroon
96
Canada
9
Chad
132
Chile
26
China
93
Colombia
43
Congo (Brazzaville)
103
Congo (Kinshasa)
127
Costa Rica
12
Croatia
75
Cyprus
49
Czech Republic
20
Denmark
2
Dominican Republic
77
Ecuador
50
Egypt
137
El Salvador
35
Estonia
55
Ethiopia
134
Finland
1
France
24
Gabon
104
Georgia
119
Germany
17
Ghana
98
Greece
82
Guatemala
27
Guinea
118
Haiti
147
Honduras
59
Hong Kong
76
Hungary
62
Iceland
4
India
140
Indonesia
92
Iran
117
Iraq
126
Ireland
16
Israel
13
Italy
36
Ivory Coast
99
Jamaica
56
Japan
58
Jordan
101
Kazakhstan
60
Kenya
121
Kosovo
46
Kuwait
51
Kyrgyzstan
86
Latvia
53
Lebanon
91
Liberia
141
Libya
72
Lithuania
42
Luxembourg
14
Madagascar
143
Malawi
150
Malaysia
80
Mali
128
Malta
22
Mauritania
122

In [86]:
#creating dictionary with data pulled from database for 2015 
d2015_dict={"country": ct_2015,
              "happiness_rank": hr_2015}

data_2015_df= pd.DataFrame(d2015_dict)

 
print(data_2015_df.shape)



#creating dictionary with data pulled from database for 2016
d2016_dict={"country": ct_2016,
              "happiness_rank": hr_2016}

data_2016_df= pd.DataFrame(d2016_dict)

 
print(data_2016_df.shape)



#creating dictionary with data pulled from database for 2017
d2017_dict={"country": ct_2017,
              "happiness_rank": hr_2017}

data_2017_df= pd.DataFrame(d2017_dict)

 
print(data_2017_df.shape)


#creating dictionary with data pulled from database for 2018
d2018_dict={"country": ct_2018,
              "happiness_rank": hr_2018}

data_2018_df= pd.DataFrame(d2018_dict)

 
print(data_2018_df.shape)


#creating dictionary with data pulled from database for 2019
d2019_dict={"country": ct_2019,
              "happiness_rank": hr_2019}

data_2019_df= pd.DataFrame(d2019_dict)

 
print(data_2019_df.shape)


(584, 2)
(438, 2)
(438, 2)
(438, 2)
(438, 2)


In [87]:
# deleting duplicates to 2015 df
data_2015_df.drop_duplicates()

# deleting duplicates to 2016 df
data_2016_df.drop_duplicates()

# deleting duplicates to 2017 df
data_2017_df.drop_duplicates()

# deleting duplicates to 2018 df
data_2018_df.drop_duplicates()

# deleting duplicates to 2019 df
data_2019_df.drop_duplicates()

In [88]:
# merging the lat and lng to 2015 df
data_2015_df = data_2015_df.merge(country_df, left_on='country', right_on='country')

# merging the lat and lng to 2016 df
data_2016_df = data_2016_df.merge(country_df, left_on='country', right_on='country')

# merging the lat and lng to 2017 df
data_2017_df = data_2017_df.merge(country_df, left_on='country', right_on='country')

# merging the lat and lng to 2018 df
data_2018_df = data_2018_df.merge(country_df, left_on='country', right_on='country')

# merging the lat and lng to 2019 df
data_2019_df = data_2019_df.merge(country_df, left_on='country', right_on='country')

In [89]:
# removing duplicates and resetting index for 2015
data_2015_df = data_2015_df.drop_duplicates()
data_2015_df.reset_index(drop=True, inplace=True)

# removing duplicates and resetting index for 2016
data_2016_df = data_2016_df.drop_duplicates()
data_2016_df.reset_index(drop=True, inplace=True)

# removing duplicates and resetting index for 2017
data_2017_df = data_2017_df.drop_duplicates()
data_2017_df.reset_index(drop=True, inplace=True)

# removing duplicates and resetting index for 2018
data_2018_df = data_2018_df.drop_duplicates()
data_2018_df.reset_index(drop=True, inplace=True)

# removing duplicates and resetting index for 2019
data_2019_df = data_2019_df.drop_duplicates()
data_2019_df.reset_index(drop=True, inplace=True)

In [90]:

# checking shape of df to make sure duplicates are gone
print(data_2015_df.shape)
print(data_2016_df.shape)
print(data_2017_df.shape)
print(data_2018_df.shape)
print(data_2019_df.shape)

(146, 4)
(146, 4)
(146, 4)
(146, 4)
(146, 4)


In [98]:

# creating json files for java script app 

d = data_2015_df.to_dict(orient="index")

with open('data_2015.json', 'w') as fp:
    json.dump(d, fp, indent=4)
    
d16 = data_2016_df.to_dict(orient="index")

with open('data_2016.json', 'w') as fp:
    json.dump(d16, fp, indent=4)
    
    
d17 = data_2017_df.to_dict(orient="index")

with open('data_2017.json', 'w') as fp:
    json.dump(d17, fp, indent=4)

    
d18 = data_2018_df.to_dict(orient="index")

with open('data_2018.json', 'w') as fp:
    json.dump(d18, fp, indent=4)
    

d19 = data_2019_df.to_dict(orient="index")

with open('data_2019.json', 'w') as fp:
    json.dump(d19, fp, indent=4)
    
    
# turning the df into the json format
# data_2015_json = data_2015_df.to_json(orient='records')
# data_2016_json = data_2016_df.to_json(orient='records')
# data_2017_json = data_2017_df.to_json(orient='records')
# data_2018_json = data_2018_df.to_json(orient='records')
# data_2019_json = data_2019_df.to_json(orient='records')


# saving the files as a json file

# with open('data_2015_json', 'r') as f:
#     json.dump(data_2015_json, f)
    
# with open('data_2016_json', 'w') as f:
#     json.dump(data_2016_json, f)
    
    
# with open('data_2017_json', 'w') as f:
#     json.dump(data_2017_json, f)
    
    
# with open('data_2018_json', 'w') as f:
#     json.dump(data_2018_json, f)
    
    
# with open('data_2019_json', 'w') as f:
#     json.dump(data_2019_json, f)
    
